In [1]:
#skip = True
skip = False
google_colab = False
import binarybrain as bb

# Google Colab用事前準備

In [2]:
# install
try:
    import binarybrain as bb
except:
    if google_colab and False:
        !pip install pybind11
        %cd /content
        !nvcc -V
        !sudo rm -fr BinaryBrain
        !rm -fr BinaryBrain
        !git clone --recursive -b ver3_develop https://github.com/ryuz/BinaryBrain.git
        %cd /content/BinaryBrain/python
        !sh copy_src.sh
        !python3 setup.py build
        !python3 setup.py develop

        import binarybrain as bb

In [3]:
# mount
if google_colab:
  from google.colab import drive 
  drive.mount('/content/drive')
  %cd /content/drive/My Drive/git-work/BinaryBrain_ver3_develop/tests/python

In [4]:
import binarybrain as bb
prop = bb.get_device_properties()
print(prop)


name                     : GeForce GT 1030
totalGlobalMem           : 2147483648
sharedMemPerBlock        : 49152
regsPerBlock             : 65536
warpSize                 : 32
memPitch                 : 2147483647
maxThreadsPerBlock       : 1024
maxThreadsDim[0]         : 1024
maxThreadsDim[1]         : 1024
maxThreadsDim[2]         : 64
maxGridSize[0]           : 2147483647
maxGridSize[1]           : 65535
maxGridSize[2]           : 65535
clockRate                : 1468000
totalConstMem            : 65536
major                    : 6
minor                    : 1
textureAlignment         : 512
deviceOverlap            : 1
multiProcessorCount      : 3
kernelExecTimeoutEnabled : 1
integrated               : 0
canMapHostMemory         : 1
computeMode              : 0




# メインコード

In [5]:
import binarybrain as bb
import numpy as np
import os
import sys
from tqdm import tqdm
from collections import OrderedDict

In [6]:
# load MNIST data
td = bb.load_mnist()

## 元レイヤー

In [7]:
modulation_size = 4
layer_rel2bin = bb.RealToBinary.create(modulation_size, framewise=True)
layer_bin2rel = bb.BinaryToReal.create(modulation_size)

# create layer
ref_layer0_affine  = bb.DenseAffine.create([32])
ref_layer0_norm    = bb.BatchNormalization.create()
ref_layer0_bin     = bb.Binarize.create()
ref_layer1_affine  = bb.DenseAffine.create([32])
ref_layer1_norm    = bb.BatchNormalization.create()
ref_layer1_bin     = bb.Binarize.create()
ref_layer2_affine  = bb.DenseAffine.create([64])
ref_layer2_norm    = bb.BatchNormalization.create()
ref_layer2_bin     = bb.Binarize.create()
ref_layer3_affine  = bb.DenseAffine.create([64])
ref_layer3_norm    = bb.BatchNormalization.create()
ref_layer3_bin     = bb.Binarize.create()
ref_layer4_affine  = bb.DenseAffine.create([512])
ref_layer4_norm    = bb.BatchNormalization.create()
ref_layer4_bin     = bb.Binarize.create()
ref_layer5_affine  = bb.DenseAffine.create([10])
ref_layer5_norm    = bb.BatchNormalization.create()
ref_layer5_bin     = bb.Binarize.create()

# main network
ref_cnv0_sub = bb.Sequential.create()
ref_cnv0_sub.add(ref_layer0_affine)
ref_cnv0_sub.add(ref_layer0_norm)
ref_cnv0_sub.add(ref_layer0_bin)
ref_layer0_cnv = bb.LoweringConvolution.create(ref_cnv0_sub, 3, 3)

ref_cnv1_sub = bb.Sequential.create()
ref_cnv1_sub.add(ref_layer1_affine)
ref_cnv1_sub.add(ref_layer1_norm)
ref_cnv1_sub.add(ref_layer1_bin)
ref_layer1_cnv = bb.LoweringConvolution.create(ref_cnv1_sub, 3, 3)

ref_cnv2_sub = bb.Sequential.create()
ref_cnv2_sub.add(ref_layer2_affine)
ref_cnv2_sub.add(ref_layer2_norm)
ref_cnv2_sub.add(ref_layer2_bin)
ref_layer2_cnv = bb.LoweringConvolution.create(ref_cnv2_sub, 3, 3)

ref_cnv3_sub = bb.Sequential.create()
ref_cnv3_sub.add(ref_layer3_affine)
ref_cnv3_sub.add(ref_layer3_norm)
ref_cnv3_sub.add(ref_layer3_bin)
ref_layer3_cnv = bb.LoweringConvolution.create(ref_cnv3_sub, 3, 3)

ref_net = bb.Sequential.create()
ref_net.add(ref_layer0_cnv)
ref_net.add(ref_layer1_cnv)
ref_net.add(bb.MaxPooling.create(2, 2))
ref_net.add(ref_layer2_cnv)
ref_net.add(ref_layer3_cnv)
ref_net.add(bb.MaxPooling.create(2, 2))
ref_net.add(ref_layer4_affine)
ref_net.add(ref_layer4_norm)
ref_net.add(ref_layer4_bin)
ref_net.add(ref_layer5_affine)
ref_net.add(ref_layer5_norm)
ref_net.add(ref_layer5_bin)

# wrapping with binary modulator
ref_train_net = bb.Sequential.create()
ref_train_net.add(layer_rel2bin)
ref_train_net.add(ref_net)
ref_train_net.add(layer_bin2rel)
ref_train_net.set_input_shape(td['x_shape'])

# Load
net_path = 'mnist-dense-cnn-binary'

if True:
    ref_layer0_affine.load_json(os.path.join(net_path, 'ref_layer0_affine.json'))
    ref_layer1_affine.load_json(os.path.join(net_path, 'ref_layer1_affine.json'))
    ref_layer2_affine.load_json(os.path.join(net_path, 'ref_layer2_affine.json'))
    ref_layer3_affine.load_json(os.path.join(net_path, 'ref_layer3_affine.json'))
    ref_layer4_affine.load_json(os.path.join(net_path, 'ref_layer4_affine.json'))
    ref_layer5_affine.load_json(os.path.join(net_path, 'ref_layer5_affine.json'))

    ref_layer0_norm.load_json(os.path.join(net_path, 'ref_layer0_norm.json'))
    ref_layer1_norm.load_json(os.path.join(net_path, 'ref_layer1_norm.json'))
    ref_layer2_norm.load_json(os.path.join(net_path, 'ref_layer2_norm.json'))
    ref_layer3_norm.load_json(os.path.join(net_path, 'ref_layer3_norm.json'))
    ref_layer4_norm.load_json(os.path.join(net_path, 'ref_layer4_norm.json'))
    ref_layer5_norm.load_json(os.path.join(net_path, 'ref_layer5_norm.json'))

In [8]:
loss      = bb.LossSoftmaxCrossEntropy.create()
metrics   = bb.MetricsCategoricalAccuracy.create()
optimizer = bb.OptimizerAdam.create()
optimizer.set_variables(ref_train_net.get_parameters(), ref_train_net.get_gradients())

if not skip:
    runner = bb.Runner(ref_train_net, "mnist-dense-cnn-binary", loss, metrics, optimizer)
    runner.fitting(td, epoch_size=1, mini_batch_size=32, file_write=False, file_read=False)

epoch=1 test_accuracy=0.992200 test_loss=1.468792 train_accuracy=0.994433 train_loss=1.465936


In [9]:
if True:
    ref_layer0_affine.save_json(os.path.join(net_path, 'ref_layer0_affine.json'))
    ref_layer1_affine.save_json(os.path.join(net_path, 'ref_layer1_affine.json'))
    ref_layer2_affine.save_json(os.path.join(net_path, 'ref_layer2_affine.json'))
    ref_layer3_affine.save_json(os.path.join(net_path, 'ref_layer3_affine.json'))
    ref_layer4_affine.save_json(os.path.join(net_path, 'ref_layer4_affine.json'))
    ref_layer5_affine.save_json(os.path.join(net_path, 'ref_layer5_affine.json'))

    ref_layer0_norm.save_json(os.path.join(net_path, 'ref_layer0_norm.json'))
    ref_layer1_norm.save_json(os.path.join(net_path, 'ref_layer1_norm.json'))
    ref_layer2_norm.save_json(os.path.join(net_path, 'ref_layer2_norm.json'))
    ref_layer3_norm.save_json(os.path.join(net_path, 'ref_layer3_norm.json'))
    ref_layer4_norm.save_json(os.path.join(net_path, 'ref_layer4_norm.json'))
    ref_layer5_norm.save_json(os.path.join(net_path, 'ref_layer5_norm.json'))

## 蒸留先

In [10]:
lut_layer0_sl0 = bb.SparseLut6.create([32])
lut_layer0_sl1 = bb.SparseLut6.create([32*6])

lut_layer1_sl0 = bb.SparseLut6.create([32])
lut_layer1_sl1 = bb.SparseLut6.create([32*6])

lut_layer2_sl0 = bb.SparseLut6.create([64])
lut_layer2_sl1 = bb.SparseLut6.create([64*6])
lut_layer2_sl2 = bb.SparseLut6.create([64*6*6])

lut_layer3_sl0 = bb.SparseLut6.create([64])
lut_layer3_sl1 = bb.SparseLut6.create([64*6])
lut_layer3_sl2 = bb.SparseLut6.create([64*6*6])

lut_layer4_sl0 = bb.SparseLut6.create([512])
lut_layer4_sl1 = bb.SparseLut6.create([512*6])

lut_layer5_sl0 = bb.SparseLut6.create([10])
lut_layer5_sl1 = bb.SparseLut6.create([10*6])
lut_layer5_sl2 = bb.SparseLut6.create([10*6*6])

# main network
lut_layer0_cnv_sub = bb.Sequential.create()
lut_layer0_cnv_sub.add(lut_layer0_sl1)
lut_layer0_cnv_sub.add(lut_layer0_sl0)
lut_layer0_cnv = bb.LoweringConvolution.create(lut_layer0_cnv_sub, 3, 3)

lut_layer1_cnv_sub = bb.Sequential.create()
lut_layer1_cnv_sub.add(lut_layer1_sl1)
lut_layer1_cnv_sub.add(lut_layer1_sl0)
lut_layer1_cnv = bb.LoweringConvolution.create(lut_layer1_cnv_sub, 3, 3)

lut_layer2_cnv_sub = bb.Sequential.create()
lut_layer2_cnv_sub.add(lut_layer2_sl2)
lut_layer2_cnv_sub.add(lut_layer2_sl1)
lut_layer2_cnv_sub.add(lut_layer2_sl0)
lut_layer2_cnv = bb.LoweringConvolution.create(lut_layer2_cnv_sub, 3, 3)

lut_layer3_cnv_sub = bb.Sequential.create()
lut_layer3_cnv_sub.add(lut_layer3_sl2)
lut_layer3_cnv_sub.add(lut_layer3_sl1)
lut_layer3_cnv_sub.add(lut_layer3_sl0)
lut_layer3_cnv = bb.LoweringConvolution.create(lut_layer3_cnv_sub, 3, 3)

lut_layer4 = bb.Sequential.create()
lut_layer4.add(lut_layer4_sl1)
lut_layer4.add(lut_layer4_sl0)

lut_layer5 = bb.Sequential.create()
lut_layer5.add(lut_layer5_sl2)
lut_layer5.add(lut_layer5_sl1)
lut_layer5.add(lut_layer5_sl0)

lut_net = bb.Sequential.create()
lut_net.add(layer_rel2bin)
lut_net.add(lut_layer0_cnv)
lut_net.add(lut_layer1_cnv)
lut_net.add(bb.MaxPooling.create(2, 2))
lut_net.add(lut_layer2_cnv)
lut_net.add(lut_layer3_cnv)
lut_net.add(bb.MaxPooling.create(2, 2))
lut_net.add(lut_layer4)
lut_net.add(lut_layer5)
lut_net.add(layer_bin2rel)
lut_net.set_input_shape(td['x_shape'])
lut_net.send_command("binary true")

In [11]:
# wrapping with binary modulator
#lut_train_net = bb.Sequential.create()
#lut_train_net.add(layer_rel2bin)
#lut_train_net.add(lut_net)
#lut_train_net.add(layer_bin2rel)
#lut_train_net.set_input_shape(td['x_shape'])

#loss      = bb.LossSoftmaxCrossEntropy.create()
#metrics   = bb.MetricsCategoricalAccuracy.create()
#optimizer = bb.OptimizerAdam.create()
#optimizer.set_variables(lut_net.get_parameters(), lut_net.get_gradients())

#runner = bb.Runner(lut_net, "mnist-lut-cnn-binary", loss, metrics, optimizer)
#runner.fitting(td, epoch_size=1, mini_batch_size=16, file_write=False, file_read=False)

In [12]:
# レイヤーの蒸留
def layer_distillation(x, x_shape, target_net, ref_net, pre_net, post_net=None, epoch=8):
    x_buf = bb.FrameBuffer()
    t_buf = bb.FrameBuffer()

#    target_net.set_input_shape(ref_net.get_input_shape())
#    target_net.send_command("binary true")
#    pre_net.set_input_shape(x_shape)

    batch_size = len(x)
    max_batch_size = 32
    leave = True

    loss = bb.LossMeanSquaredError.create()
    optimizer = bb.OptimizerAdam.create()
    optimizer.set_variables(target_net.get_parameters(), target_net.get_gradients())
    
    for e in range(epoch):
        loss.clear()
        with tqdm(range(0, batch_size, max_batch_size), leave=leave) as pbar:
            for index in pbar:
                # calc mini_batch_size
                mini_batch_size = min(max_batch_size, batch_size-index)
                
                # setup x
                x_buf.resize(mini_batch_size, x_shape)
                x_buf.set_data(x[index:index+mini_batch_size])
                
                # forward
                x_buf = pre_net.forward(x_buf, False)
                t_buf = ref_net.forward(x_buf, False)
                if post_net is not None:
                    t_buf = post_net.forward(t_buf, False)
                
                y_buf = target_net.forward(x_buf, True)
                if post_net is not None:
                    y_buf = post_net.forward(y_buf, True)
                
                # calc loss
                dy_buf = loss.calculate_loss(y_buf, t_buf, mini_batch_size)

                # backward
                if post_net is not None:
                    dy_buf = post_net.backward(dy_buf)
                target_net.backward(dy_buf)
                
                # update
                optimizer.update()
                
                # print progress
                dict = OrderedDict()
                dict['loss'] = loss.get_loss()
                if len(dict) > 0:
                    pbar.set_postfix(dict)

        # shuffle
        p = np.random.permutation(len(x))
        x = x[p]

In [13]:
# 事前ネット
pre_net = bb.Sequential.create()
pre_net.add(layer_rel2bin)
pre_net.send_command("binary true")

## Layer0

In [14]:
# Layer0
tW = ref_layer0_affine.W()
W = np.array(tW.get_data()).reshape(tW.get_shape()[::-1])
idx = np.argsort(-np.abs(W), axis=1)

# Weight順で接続
if True:
    for i in range(32):
        for j in range(6):
            lut_layer0_sl0.set_connection_index([i], j, i*6+j)

    for i in range(32):
        for j in range(6):
            for k in range(6):
                lut_layer0_sl1.set_connection_index([i*6+j], k, idx[i][(j*6+k) % (3*3)])

if True:
    try:
        lut_layer0_sl0.load_json(os.path.join(net_path, 'lut_layer0_sl0.json'))
        lut_layer0_sl1.load_json(os.path.join(net_path, 'lut_layer0_sl1.json'))
    except:
        pass

In [15]:
if not skip:
    layer_distillation(td['x_train'], td['x_shape'], lut_layer0_cnv, ref_layer0_cnv, pre_net, epoch=1)

100%|████████████████████████████████████████████████████████████████| 1875/1875 [07:59<00:00,  3.91it/s, loss=0.00227]


In [16]:
if True:
    lut_layer0_sl0.save_json(os.path.join(net_path, 'lut_layer0_sl0.json'))
    lut_layer0_sl1.save_json(os.path.join(net_path, 'lut_layer0_sl1.json'))

In [17]:
# ファインチューニング
def fine_tuning(x, x_shape, t, t_shape, pre_net, train_net, epoch=2):
    x_buf = bb.FrameBuffer()
    t_buf = bb.FrameBuffer()

    batch_size = len(x)
    max_batch_size = 32
    leave = True

    loss = bb.LossMeanSquaredError.create()
    metrics = bb.MetricsCategoricalAccuracy.create()
    optimizer = bb.OptimizerAdam.create()
    optimizer.set_variables(train_net.get_parameters(), train_net.get_gradients())
    
    for e in range(epoch):
        loss.clear()
        with tqdm(range(0, batch_size, max_batch_size), leave=leave) as pbar:
            for index in pbar:
                # calc mini_batch_size
                mini_batch_size = min(max_batch_size, batch_size-index)
                
                # setup x
                x_buf.resize(mini_batch_size, x_shape)
                x_buf.set_data(x[index:index+mini_batch_size])
                
                # forward
                x_buf = pre_net.forward(x_buf, False)
                y_buf = train_net.forward(x_buf)

                # setup t
                t_buf.resize(mini_batch_size, t_shape)
                t_buf.set_data(t[index:index+mini_batch_size])

                # calc loss
                dy_buf = loss.calculate_loss(y_buf, t_buf, mini_batch_size)
                metrics.calculate_metrics(y_buf, t_buf)
                
                # backward
                train_net.backward(dy_buf)

                # update
                optimizer.update()
                
                # print progress
                dict = OrderedDict()
                dict['loss'] = loss.get_loss()
                dict[metrics.get_metrics_string()] = metrics.get_metrics()
                pbar.set_postfix(dict)

                if len(dict) > 0:
                    pbar.set_postfix(dict)
        
        # shuffle
        p = np.random.permutation(len(x))
        x = x[p]
        t = t[p]

In [18]:
fine_pre_net = bb.Sequential.create()
fine_pre_net.add(layer_rel2bin)
fine_pre_net.add(lut_layer0_cnv)

fine_net = bb.Sequential.create()
fine_net.add(ref_layer1_cnv)
fine_net.add(bb.MaxPooling.create(2, 2))
fine_net.add(ref_layer2_cnv)
fine_net.add(ref_layer3_cnv)
fine_net.add(bb.MaxPooling.create(2, 2))
fine_net.add(ref_layer4_affine)
fine_net.add(ref_layer4_norm)
fine_net.add(ref_layer4_bin)
fine_net.add(ref_layer5_affine)
fine_net.add(ref_layer5_norm)
fine_net.add(ref_layer5_bin)
fine_net.add(layer_bin2rel)

if not skip:
    fine_tuning(td['x_train'], td['x_shape'], td['t_train'], td['t_shape'], fine_pre_net, fine_net, epoch=1)

100%|████████████████████████████████████████████████| 1875/1875 [03:31<00:00,  8.88it/s, loss=0.00207, accuracy=0.992]


In [19]:
if True:
    ref_layer0_affine.save_json(os.path.join(net_path, 'fine_layer0_affine.json'))
    ref_layer1_affine.save_json(os.path.join(net_path, 'fine_layer1_affine.json'))
    ref_layer2_affine.save_json(os.path.join(net_path, 'fine_layer2_affine.json'))
    ref_layer3_affine.save_json(os.path.join(net_path, 'fine_layer3_affine.json'))
    ref_layer4_affine.save_json(os.path.join(net_path, 'fine_layer4_affine.json'))
    ref_layer5_affine.save_json(os.path.join(net_path, 'fine_layer5_affine.json'))
    
    ref_layer0_norm.save_json(os.path.join(net_path, 'fine_layer0_norm.json'))
    ref_layer1_norm.save_json(os.path.join(net_path, 'fine_layer1_norm.json'))
    ref_layer2_norm.save_json(os.path.join(net_path, 'fine_layer2_norm.json'))
    ref_layer3_norm.save_json(os.path.join(net_path, 'fine_layer3_norm.json'))
    ref_layer4_norm.save_json(os.path.join(net_path, 'fine_layer4_norm.json'))
    ref_layer5_norm.save_json(os.path.join(net_path, 'fine_layer5_norm.json'))

## Layer1

In [20]:
# Layer1
tW = ref_layer1_affine.W()
W = np.array(tW.get_data()).reshape(tW.get_shape()[::-1])
idx = np.argsort(-np.abs(W), axis=1)

In [21]:
#if True:
#    for i in range(32):
#        for j in range(6):
#            lut_layer1_sl0.set_connection_index([i], j, i*6+j)

#    for i in range(32):
#        for j in range(6*6):
#            lut_layer1_sl1.set_connection_index([i], j, idx[i][j])

# Weight順で接続
if True:
    for i in range(32):
        for j in range(6):
            lut_layer1_sl0.set_connection_index([i], j, i*6+j)

    for i in range(32):
        for j in range(6):
            for k in range(6):
                lut_layer1_sl1.set_connection_index([i*6+j], k, idx[i][(j*6+k) % (3*3*32)])

In [22]:
pre_net = bb.Sequential.create()
pre_net.add(layer_rel2bin)
pre_net.add(lut_layer0_cnv)
pre_net.send_command("binary true")

if True:
    try:
        lut_layer1_sl0.load_json(os.path.join(net_path, 'lut_layer1_sl0.json'))
        lut_layer1_sl1.load_json(os.path.join(net_path, 'lut_layer1_sl1.json'))
    except:
        pass

In [23]:
if not skip:
    layer_distillation(td['x_train'], td['x_shape'], lut_layer1_cnv, ref_layer1_cnv, pre_net, epoch=1)

100%|█████████████████████████████████████████████████████████████████| 1875/1875 [09:30<00:00,  3.29it/s, loss=0.0518]


In [24]:
if True:
    lut_layer1_sl0.save_json(os.path.join(net_path, 'lut_layer1_sl0.json'))
    lut_layer1_sl1.save_json(os.path.join(net_path, 'lut_layer1_sl1.json'))

In [25]:
fine_pre_net = bb.Sequential.create()
fine_pre_net.add(layer_rel2bin)
fine_pre_net.add(lut_layer0_cnv)
fine_pre_net.add(lut_layer1_cnv)
fine_pre_net.add(bb.MaxPooling.create(2, 2))

fine_net = bb.Sequential.create()
fine_net.add(ref_layer2_cnv)
fine_net.add(ref_layer3_cnv)
fine_net.add(bb.MaxPooling.create(2, 2))
fine_net.add(ref_layer4_affine)
fine_net.add(ref_layer4_norm)
fine_net.add(ref_layer4_bin)
fine_net.add(ref_layer5_affine)
fine_net.add(ref_layer5_norm)
fine_net.add(ref_layer5_bin)
fine_net.add(layer_bin2rel)

if not skip:
    fine_tuning(td['x_train'], td['x_shape'], td['t_train'], td['t_shape'], fine_pre_net, fine_net, epoch=1)

100%|████████████████████████████████████████████████| 1875/1875 [02:32<00:00, 12.27it/s, loss=0.00172, accuracy=0.993]


## Layer2

In [26]:
# Layer2
tW = ref_layer2_affine.W()
W = np.array(tW.get_data()).reshape(tW.get_shape()[::-1])
idx = np.argsort(-np.abs(W), axis=1)

In [27]:
# Weight順で接続
if False:
    for i in range(64):
        for j in range(6):
            lut_layer2_sl0.set_connection_index([i], j, i*6+j)

    for i in range(64):
        for j in range(6):
            for k in range(6):
                lut_layer2_sl1.set_connection_index([i*6+j], k, idx[i][(j*6+k) % (3*3*32)])

if True:
    for i in range(64):
        for j in range(6):
            lut_layer2_sl0.set_connection_index([i], j, i*6+j)
    
    for i in range(64*6):
        for j in range(6):
            lut_layer2_sl1.set_connection_index([i], j, i*6+j)
    
    for i in range(64):
        for j in range(6*6):
            for k in range(6):
                lut_layer2_sl2.set_connection_index([i*6*6+j], k, idx[i][(j*6*6+k) % (3*3*32)])

In [28]:
pre_net = bb.Sequential.create()
pre_net.add(layer_rel2bin)
pre_net.add(lut_layer0_cnv)
pre_net.add(lut_layer1_cnv)
pre_net.add(bb.MaxPooling.create(2, 2))
pre_net.send_command("binary true")

if True:
    lut_layer0_sl0.load_json(os.path.join(net_path, 'lut_layer0_sl0.json'))
    lut_layer0_sl1.load_json(os.path.join(net_path, 'lut_layer0_sl1.json'))
    lut_layer1_sl0.load_json(os.path.join(net_path, 'lut_layer1_sl0.json'))
    lut_layer1_sl1.load_json(os.path.join(net_path, 'lut_layer1_sl1.json'))

if True:
    try:
        lut_layer2_sl0.load_json(os.path.join(net_path, 'lut_layer2_sl0.json'))
        lut_layer2_sl1.load_json(os.path.join(net_path, 'lut_layer2_sl1.json'))
        lut_layer2_sl2.load_json(os.path.join(net_path, 'lut_layer2_sl2.json'))
    except:
        pass

if not skip:
    tmp_bin2rel = bb.BinaryToReal.create(modulation_size)
    tmp_bin2rel.set_input_shape(lut_layer2_cnv.get_output_shape())
#   layer_distillation(td['x_train'], td['x_shape'], lut_layer2_cnv, ref_layer2_cnv, pre_net, tmp_bin2rel, epoch=1)
    layer_distillation(td['x_train'], td['x_shape'], lut_layer2_cnv, ref_layer2_cnv, pre_net, epoch=1)

100%|██████████████████████████████████████████████████████████████████| 1875/1875 [09:41<00:00,  3.22it/s, loss=0.176]


In [29]:
if True:
    lut_layer2_sl0.save_json(os.path.join(net_path, 'lut_layer2_sl0.json'))
    lut_layer2_sl1.save_json(os.path.join(net_path, 'lut_layer2_sl1.json'))
    lut_layer2_sl2.save_json(os.path.join(net_path, 'lut_layer2_sl2.json'))

In [30]:
fine_pre_net = bb.Sequential.create()
fine_pre_net.add(layer_rel2bin)
fine_pre_net.add(lut_layer0_cnv)
fine_pre_net.add(lut_layer1_cnv)
fine_pre_net.add(bb.MaxPooling.create(2, 2))
fine_pre_net.add(lut_layer2_cnv)

fine_net = bb.Sequential.create()
fine_net.add(ref_layer3_cnv)
fine_net.add(bb.MaxPooling.create(2, 2))
fine_net.add(ref_layer4_affine)
fine_net.add(ref_layer4_norm)
fine_net.add(ref_layer4_bin)
fine_net.add(ref_layer5_affine)
fine_net.add(ref_layer5_norm)
fine_net.add(ref_layer5_bin)
fine_net.add(layer_bin2rel)

if not skip:
    fine_tuning(td['x_train'], td['x_shape'], td['t_train'], td['t_shape'], fine_pre_net, fine_net, epoch=1)

100%|████████████████████████████████████████████████| 1875/1875 [02:58<00:00, 10.50it/s, loss=0.00331, accuracy=0.985]


## Layer3

In [31]:
# Layer3
tW = ref_layer3_affine.W()
W = np.array(tW.get_data()).reshape(tW.get_shape()[::-1])
idx = np.argsort(-np.abs(W), axis=1)

In [32]:
if True:
    for i in range(64):
        for j in range(6):
            lut_layer3_sl0.set_connection_index([i], j, i*6+j)
    
    for i in range(64*6):
        for j in range(6):
            lut_layer3_sl1.set_connection_index([i], j, i*6+j)
    
    for i in range(64):
        for j in range(6*6):
            for k in range(6):
                lut_layer3_sl2.set_connection_index([i*6*6+j], k, idx[i][(j*6*6+k) % (3*3*64)])

In [36]:
pre_net = bb.Sequential.create()
pre_net.add(layer_rel2bin)
pre_net.add(lut_layer0_cnv)
pre_net.add(lut_layer1_cnv)
pre_net.add(bb.MaxPooling.create(2, 2))
pre_net.add(lut_layer2_cnv)
pre_net.send_command("binary true")

if True:
    lut_layer0_sl0.load_json(os.path.join(net_path, 'lut_layer0_sl0.json'))
    lut_layer0_sl1.load_json(os.path.join(net_path, 'lut_layer0_sl1.json'))
    lut_layer1_sl0.load_json(os.path.join(net_path, 'lut_layer1_sl0.json'))
    lut_layer1_sl1.load_json(os.path.join(net_path, 'lut_layer1_sl1.json'))
    lut_layer2_sl0.load_json(os.path.join(net_path, 'lut_layer2_sl0.json'))
    lut_layer2_sl1.load_json(os.path.join(net_path, 'lut_layer2_sl1.json'))
    lut_layer2_sl2.load_json(os.path.join(net_path, 'lut_layer2_sl2.json'))

if True:
    try:
        lut_layer3_sl0.load_json(os.path.join(net_path, 'lut_layer3_sl0.json'))
        lut_layer3_sl1.load_json(os.path.join(net_path, 'lut_layer3_sl1.json'))
        lut_layer3_sl2.load_json(os.path.join(net_path, 'lut_layer3_sl2.json'))
    except:
        pass

#if not skip:
if True:
    tmp_bin2rel = bb.BinaryToReal.create(modulation_size)
    tmp_bin2rel.set_input_shape(lut_layer3_cnv.get_output_shape())
    #layer_distillation(td['x_train'], td['x_shape'], lut_layer3_cnv, ref_layer3_cnv, pre_net, tmp_bin2rel, epoch=2)
    layer_distillation(td['x_train'], td['x_shape'], lut_layer3_cnv, ref_layer3_cnv, pre_net, epoch=1)

100%|██████████████████████████████████████████████████████████████████| 1875/1875 [07:29<00:00,  4.17it/s, loss=0.334]


In [34]:
if True:
    lut_layer3_sl0.save_json(os.path.join(net_path, 'lut_layer3_sl0.json'))
    lut_layer3_sl1.save_json(os.path.join(net_path, 'lut_layer3_sl1.json'))
    lut_layer3_sl2.save_json(os.path.join(net_path, 'lut_layer3_sl2.json'))

In [35]:
fine_pre_net = bb.Sequential.create()
fine_pre_net.add(layer_rel2bin)
fine_pre_net.add(lut_layer0_cnv)
fine_pre_net.add(lut_layer1_cnv)
fine_pre_net.add(bb.MaxPooling.create(2, 2))
fine_pre_net.add(lut_layer2_cnv)
fine_pre_net.add(lut_layer3_cnv)
fine_pre_net.add(bb.MaxPooling.create(2, 2))

fine_net = bb.Sequential.create()
fine_net.add(ref_layer4_affine)
fine_net.add(ref_layer4_norm)
fine_net.add(ref_layer4_bin)
fine_net.add(ref_layer5_affine)
fine_net.add(ref_layer5_norm)
fine_net.add(ref_layer5_bin)
fine_net.add(layer_bin2rel)

#if not skip:
if True:
    fine_tuning(td['x_train'], td['x_shape'], td['t_train'], td['t_shape'], fine_pre_net, fine_net, epoch=1)

100%|██████████████████████████████████████████████████| 1875/1875 [03:05<00:00, 10.12it/s, loss=0.015, accuracy=0.924]
